In [6]:
import pandas as pd
import easyocr
import os
from tqdm.notebook import tqdm
import torch
import numpy as np
import cv2

rootPath = "D:/DTU - D/4/Fagprojekt/Data"
csvName = "labels.csv"
reader = easyocr.Reader(['en'])
data = pd.read_csv(os.path.join(rootPath,csvName))
batch = 1
data.drop(1711, inplace=True) #GIF
rows = data.shape[0]

In [2]:
results = []
ground_truth = []
resolutions = np.zeros(shape=(rows,2))
correctIn = 0
for i in tqdm(range(rows)):
    # Primitive attempt at batching
    if i % batch == 0:
        del reader
        torch.cuda.empty_cache()
        reader = easyocr.Reader(['en'])

    path = os.path.join(rootPath,data.iloc[i]['image_path'])

    img = cv2.imread(path)
    resolutions[i] = [img.shape[0],img.shape[1]]

    result = reader.readtext(path)
    results.append(result)

    y = data.iloc[i]['meter_number']
    ground_truth.append(y)
    for j in range(len(result)):
        if y in result[j]:
            correctIn += 1
            break

  0%|          | 0/2061 [00:00<?, ?it/s]

C:\Users\Damho\anaconda3\lib\site-packages\PIL\Image.py:2855: DecompressionBombWarning: Image size (108000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


In [3]:
# # Rename files
# tochange = [29,45,133,135,136,180,186,203,303,347,349,369,379,429,514,562,566,644,645,713,821,946,1008,1073,1074,1087,1211,1212,1233,1325,1558,1568,1573,1574,1577,1611,1743,1900,372,908,958,959,992,993,1061,1362,1651,1671,1675,1735]
#
# for i in tochange:
#     image_path = data.iloc[i]['image_path']
#     newName = os.path.join(rootPath,image_path.replace("å","aa").replace("ø","oo").replace("æ","ae"))
#     oldFileName = os.listdir(os.path.join(rootPath,os.path.dirname(image_path)))[0]
#     oldName = os.path.join(rootPath,os.path.dirname(image_path),oldFileName)
#     os.rename(oldName,newName)

In [96]:
# np.savez("Variables_results_17_03_2022_detailsOn",
#          correctIn=correctIn,
#          ground_truth=ground_truth,
#          resolutions=resolutions,
#          results=np.array(results,dtype=object),
#          unique_reso = unique_reso)

In [5]:
# import numpy as np
# wut = np.load("D:/DTU - D/4/Fagprojekt/NoteBook_project/dsProject/Variables_results_17_03_2022_detailsOn.npz")
# resolutions = wut["resolutions"]

In [6]:
# Get unique resolutions
unique_reso = []
unique_reso.append(resolutions[0])
for i in range(len(resolutions)):
    unique = True
    for j in range(len(unique_reso)):
        if resolutions[i][0] == unique_reso[j][0] and resolutions[i][1] == unique_reso[j][1]:
            unique = False
            break
    if unique:
        unique_reso.append(resolutions[i])

In [7]:
# Get count on unique resolutions
def getCount(array, toLook):
    count = 0
    for i in range(len(toLook)):
        if (array==toLook[i]).all():
            count += 1
    return count

dict_unique = []
for i in range(len(unique_reso)):
    dict_unique.append((unique_reso[i],getCount(unique_reso[i], resolutions)))
reso_count = np.array(dict_unique)

C:\Users\Damho\AppData\Local\Temp/ipykernel_46804/2467117560.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  reso_count = np.array(dict_unique)


In [106]:
print(f"Number of pictures: {rows}")
print(f"number of unique resolutions: {len(reso_count)}")
print(f"Number of resolutions with more than 5 pictures: {np.sum(reso_count[:,1]>5)}")
print(f"Number of resolutions with more than 10 pictures: {np.sum(reso_count[:,1]>10)}")
print(f"Number of resolutions with more than 15 pictures: {np.sum(reso_count[:,1]>15)}")
print(f"Number of resolutions with more than 20 pictures: {np.sum(reso_count[:,1]>20)}")
print(f"Number of resolutions with more than 40 pictures: {np.sum(reso_count[:,1]>40)}")
print(f"pretrained accuracy: {(correctIn/len(data.project_id.unique()))*100}")

Number of pictures: 2061
number of unique resolutions: 225
Number of resolutions with more than 5 pictures: 34
Number of resolutions with more than 10 pictures: 21
Number of resolutions with more than 15 pictures: 17
Number of resolutions with more than 20 pictures: 12
Number of resolutions with more than 40 pictures: 8
pretrained accuracy: 8.967596081386587


In [8]:
to_check = reso_count[np.argsort(reso_count[:,1])][-8:,:]
for i in range(len(to_check)):
    idxs = [j for j,x in enumerate(resolutions) if x[0] == to_check[i][0][0] and x[1] == to_check[i][0][1]]
    result = [results[i] for i in idxs]

NameError: name 'results' is not defined